# We need to install the dependencies

In [1]:
!git clone https://github.com/AI4Bharat/NeMo.git && cd NeMo && git checkout nemo-v2 && bash reinstall.sh
!pip install onnxruntime
!pip install onnx
!pip install onnxmltools

!wget https://github.com/deepanshu-yadav/Quantize_speech_Recognition_For_Hindi/raw/refs/heads/main/file.wav

Cloning into 'NeMo'...
remote: Enumerating objects: 79519, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 79519 (delta 186), reused 178 (delta 162), pack-reused 79295 (from 3)
Receiving objects: 100% (79519/79519), 141.13 MiB | 9.31 MiB/s, done.
Resolving deltas: 100% (57717/57717), done.
Already on 'nemo-v2'
Your branch is up to date with 'origin/nemo-v2'.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Uninstalling stuff
Installing nemo
Obtaining file:///content/NeMo
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... 

In [2]:
!pip uninstall numpy -y
!pip install numpy==1.26
!pip install onnxsim
!pip install onnxconverter-common

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 65.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.0 which is incompatible.
dask-cuda 25.2.0 requires click>=8.1, but you have click 8.0.2 which is incompatible.
orbax-checkpoint 0.11.15 requires tensorstore>=0.1.71, but you have tensorstore 0.1.45 which is incompatible.
peft 0.15.2 requires huggingface_hub>=0.25.0, but you have huggingface-hub 0.23.2 which is incompatible.
gradio 5.31.0 requires huggingface-hub>=0.28.1, but you have huggingface-hub 0.23.2 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 39.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of onnx to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 121.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: onnx
    Found existing installation: onnx 1.18.0
    Uninstalling onnx-1.18.0:
      Successfully uninstalled onnx-1.18.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [onnxconverter-common]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.2 which is incompatible.
orbax-checkpoint 0.11.15 requires tensorstore>=0.

## Restart the session after this

In [2]:
import onnx
from onnx import shape_inference

def print_model(file_name):
  # Load and check the ONNX model
  onnx_model = onnx.load(file_name)
  onnx.checker.check_model(onnx_model)

  # Run shape inference to populate shapes
  inferred_model = shape_inference.infer_shapes(onnx_model)

  # Function to extract input/output shapes
  def get_io_shapes(model):
      input_shapes = {}
      output_shapes = {}

      for input_tensor in model.graph.input:
          shape = [dim.dim_value if dim.dim_value > 0 else "?" for dim in input_tensor.type.tensor_type.shape.dim]
          input_shapes[input_tensor.name] = shape

      for output_tensor in model.graph.output:
          shape = [dim.dim_value if dim.dim_value > 0 else "?" for dim in output_tensor.type.tensor_type.shape.dim]
          output_shapes[output_tensor.name] = shape

      return input_shapes, output_shapes

  # Get and print input/output shapes
  input_shapes, output_shapes = get_io_shapes(inferred_model)

  print("Inputs:")
  for name, shape in input_shapes.items():
      print(f"  {name}: {shape}")

  print("\nOutputs:")
  for name, shape in output_shapes.items():
      print(f"  {name}: {shape}")


In [16]:
from pathlib import Path
from typing import Dict
import os
import nemo.collections.asr as nemo_asr
import onnx
import torch
import torch.nn as nn

# Load the model
print("Loading NeMo model...")
# Using a specific revision for reproducibility
model = nemo_asr.models.ASRModel.from_pretrained(
    "ai4bharat/indicconformer_stt_hi_hybrid_rnnt_large",
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.freeze()  # Set to inference mode
model = model.to(device)
model.eval()
print("✅ Model loaded successfully.")

Loading NeMo model...
[NeMo I 2025-06-18 18:51:07 nemo_logging:381] _setup_tokenizer: detected an aggregate tokenizer
[NeMo I 2025-06-18 18:51:07 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2025-06-18 18:51:07 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2025-06-18 18:51:07 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2025-06-18 18:51:07 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2025-06-18 18:51:07 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2025-06-18 18:51:07 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2025-06-18 18:51:07 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2025-06-18 18:51:07 nemo_logging:381] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2025-06-18 18:51:0

[NeMo W 2025-06-18 18:51:14 nemo_logging:393] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath:
    - /nlsasfs/home/ai4bharat/ai4bharat-pr/speechteam/indicasr_v3/manifests/nemo/vistaar_v3/train/train_hindi.json
    sample_rate: 16000
    batch_size: 8
    num_workers: 16
    pin_memory: true
    max_duration: 30.0
    min_duration: 0.2
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    is_concat: true
    concat_sampling_technique: temperature
    concat_sampling_temperature: 1.5
    return_language_id: true
    
[NeMo W 2025-06-18 18:51:14 nemo_logging:393] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to 

[NeMo I 2025-06-18 18:51:14 nemo_logging:381] PADDING: 0
[NeMo I 2025-06-18 18:51:18 nemo_logging:381] Vocab size for each language: 256
[NeMo I 2025-06-18 18:51:18 nemo_logging:381] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-06-18 18:51:18 nemo_logging:381] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-06-18 18:51:20 nemo_logging:381] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-06-18 18:51:20 nemo_logging:381] Creating masks for multi-softmax layer.
[NeMo I 2025-06-18 18:51:20 nemo_logging:381] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-06-18 18:51:21 nemo_logging:381] Model EncDecHybridRNNTCTCBPEModel was successfully restored from /root/.cache/huggingface/hub/models--ai4bharat--indicconformer

In [17]:
from pathlib import Path
from typing import Dict
import os
import nemo.collections.asr as nemo_asr
import onnx
import torch
import torch.nn as nn


def add_meta_data(filename: str, meta_data: Dict[str, str]):
    """Add meta data to an ONNX model. It is changed in-place.

    Args:
      filename:
        Filename of the ONNX model to be changed.
      meta_data:
        Key-value pairs.
    """
    model = onnx.load(filename)
    while len(model.metadata_props):
        model.metadata_props.pop()

    for key, value in meta_data.items():
        meta = model.metadata_props.add()
        meta.key = key
        meta.value = str(value)

    onnx.save(model, filename)

# Custom wrapper for joint_net to define forward method
class JointNetWrapper(nn.Module):
    def __init__(self, joint_module, lang_ids):
        super(JointNetWrapper, self).__init__()
        self.joint_module = joint_module
        self.lang_ids = lang_ids

    def forward(self, encoder_outputs, decoder_outputs):
        encoder_outputs = encoder_outputs.transpose(1, 2)  # (B, T, D)
        decoder_outputs = decoder_outputs.transpose(1, 2)
        return self.joint_module.joint(encoder_outputs, decoder_outputs, self.lang_ids)


@torch.no_grad()
def main():
    # Save tokens
    with open("./tokens.txt", "w", encoding="utf-8") as f:
      for i, s in enumerate(model.joint.vocabulary):
          f.write(f"{s} {i}\n")
      f.write(f"<blk> {i+1}\n")
    print("Saved to tokens.txt")
    model.set_export_config({"decoder_type": "rnnt", "cache_support": False})
    # Export encoder and decoder
    model.encoder.export("encoder.onnx")
    model.decoder.export("decoder.onnx")
    print("✅ Encoder and decoder exported.")
    model.joint._prepare_for_export()
    # Wrap joint_net for export
    # joint_wrapper = JointNetWrapper(model.joint, model.joint.language_keys)
    joint_wrapper = JointNetWrapper(model.joint, ["hi"])
    joint_wrapper.eval()

    # max_batch = 1
    # max_dim = 1
    # B, T, U = max_batch, max_dim, max_batch
    # encoder_outputs = torch.randn(B, 512, T).to(device)
    # decoder_outputs = torch.randn(B, 640, U).to(device)
    # input_example =  (encoder_outputs, decoder_outputs)
    # print(input_example[0].shape, input_example[1].shape)
    input_example = model.joint.input_example()
    print(input_example[0].shape, input_example[1].shape)
    # Export joint network to ONNX
    torch.onnx.export(
        joint_wrapper,
        #(encoder_output, decoder_output),
        input_example,
        "joiner.onnx",
        input_names=["encoder_outputs", "decoder_outputs"],
        output_names=["output"],
        dynamic_axes={
        "encoder_outputs": {0: "batch_size", 2: "encoder_seq_len"},  # First and third dimensions
        "decoder_outputs": {0: "batch_size", 2: "decoder_seq_len"},  # First and third dimensions
        "output": {0: "batch_size"}  # Output batch dimension
        },
        opset_version=13, # Using a stable opset version
        verbose=False,
    )
    print("✅ Joint network exported to joiner.onnx")

    # Print file sizes
    os.system("ls -lh *.onnx")
    normalize_type = model.cfg.preprocessor.normalize
    if normalize_type == "NA":
      normalize_type = ""
    meta_data = {
    "vocab_size": model.decoder.vocab_size,
    "normalize_type": normalize_type,
    "pred_rnn_layers": model.decoder.pred_rnn_layers,
    "pred_hidden": model.decoder.pred_hidden,
    "subsampling_factor": 8,
    "model_type": "EncDecRNNTBPEModel",
    "version": "2",
    "model_author": "AI4Bharat",
    "url": "https://huggingface.co/ai4bharat/indicconformer_stt_hi_hybrid_ctc_rnnt_large",
    "comment": "Only the transducer branch is exported",
    "feat_dim": 80,
    }
    add_meta_data("encoder.onnx", meta_data)

if __name__ == "__main__":
    main()

Saved to tokens.txt
[NeMo I 2025-06-18 18:51:33 nemo_logging:381] No `decoding_cfg` passed when changing decoding strategy, using internal config
[NeMo I 2025-06-18 18:51:33 nemo_logging:381] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-06-18 18:51:33 nemo_logging:381] Changed decoding strategy of the RNNT decoder to 
    model_type: rnnt
    strategy: greedy_batch
    compute_hypothesis_token_set: false
    preserve_alignments: null
    confidence_cfg:
      preserve_frame_confidence: false
      preserve_token_confidence: false
      preserve_word_confidence: false
      exclude_blank: true
      aggregation: min
      method_cfg:
        name: entropy
        entropy_type: tsallis
        alpha: 0.33
        entropy_norm: exp
        temperature: DEPRECATED
    fused_batch_size: null
    compute_timestamps: null
    compute_langs: false
    word_seperator: ' '
    rnnt_timestamp_type: all
    greedy:
      max_

In [18]:
# input_example = model.joint.input_example()
# model.joint._fuse_loss_wer = False
# joint_out = model.joint.forward(encoder_outputs=input_example[0], decoder_outputs=input_example[1], compute_wer=False,language_ids=model.joint.language_keys)
# print(joint_out.shape)

In [19]:
print("Print Decoder")
print_model("decoder.onnx")
print("-"*80)
print("Enocder contents")
print_model("encoder.onnx")
print("-"*80)
print("Joint cotents")
print_model("joiner.onnx")

Print Decoder
Inputs:
  targets: ['?', '?']
  target_length: ['?']
  states.1: [1, '?', 640]
  onnx::LSTM_3: [1, 1, 640]

Outputs:
  outputs: ['?', 640, '?']
  prednet_lengths: ['?']
  states: [1, '?', 640]
  74: [1, '?', 640]
--------------------------------------------------------------------------------
Enocder contents
Inputs:
  audio_signal: ['?', 80, '?']
  length: ['?']

Outputs:
  outputs: ['?', 512, '?']
  encoded_lengths: ['?']
--------------------------------------------------------------------------------
Joint cotents
Inputs:
  encoder_outputs: ['?', 512, '?']
  decoder_outputs: ['?', 640, '?']

Outputs:
  output: ['?', '?', '?', 257]
Inputs:
  targets: ['?', '?']
  target_length: ['?']
  states.1: [2, '?', 640]
  onnx::Slice_3: [2, 1, 640]

Outputs:
  outputs: ['?', 640, '?']
  prednet_lengths: ['?']
  states: [2, '?', 640]
  162: [2, '?', 640]
Inputs:
  audio_signal: ['?', 128, '?']
  length: ['?']

Outputs:
  outputs: ['?', 1024, '?']
  encoded_lengths: ['?']
Inputs:
  

In [20]:
import os
import json
import numpy as np
import onnxruntime as ort
import soundfile as sf
import torch
import torchaudio
import torchaudio.transforms as T
from librosa import resample
from typing import Tuple

class StandaloneASR:
    def __init__(self, model_dir: str = "model_components"):
        """Initialize the transducer-based ASR system."""
        self.model_dir = model_dir
        self.device = "cuda" if ort.get_device() == "GPU" else "cpu"

        # Ensure model_dir exists
        if not os.path.exists(model_dir):
            raise FileNotFoundError(f"Model directory {model_dir} does not exist")

        # Load ONNX models
        encoder_path = os.path.join(model_dir, "encoder.onnx")
        decoder_path = os.path.join(model_dir, "decoder.onnx")
        joiner_path = os.path.join(model_dir, "joiner.onnx")
        for path in [encoder_path, decoder_path, joiner_path]:
            if not os.path.exists(path):
                raise FileNotFoundError(f"Model file {path} not found")

        session_opts = ort.SessionOptions()
        session_opts.inter_op_num_threads = 1
        session_opts.intra_op_num_threads = 1
        providers = ["CUDAExecutionProvider" if self.device == "cuda" else "CPUExecutionProvider"]

        self.encoder = ort.InferenceSession(encoder_path, sess_options=session_opts, providers=providers)
        self.decoder = ort.InferenceSession(decoder_path, sess_options=session_opts, providers=providers)
        self.joiner = ort.InferenceSession(joiner_path, sess_options=session_opts, providers=providers)

        # Load encoder metadata
        meta = self.encoder.get_modelmeta().custom_metadata_map
        self.normalize_type = meta.get("normalize_type", "")
        self.pred_rnn_layers = int(meta.get("pred_rnn_layers", 2))
        self.pred_hidden = int(meta.get("pred_hidden", 512))

        # Load vocabulary
        vocab_path = os.path.join(model_dir, "tokens.txt")
        if not os.path.exists(vocab_path):
            raise FileNotFoundError(f"Vocab file {vocab_path} not found")
        self.id2token = {}
        with open(vocab_path, encoding="utf-8") as f:
            for line in f:
                t, idx = line.strip().split()
                self.id2token[int(idx)] = t
        self.blank_id = 256  # No_of_vocab - 1  = 257 - 1

    def get_decoder_state(self):
        """Initialize decoder states."""
        batch_size = 1
        state0 = np.zeros((self.pred_rnn_layers, batch_size, self.pred_hidden), dtype=np.float32)
        state1 = np.zeros((self.pred_rnn_layers, batch_size, self.pred_hidden), dtype=np.float32)
        return state0, state1

    def calculate_audio_features(self, audio: np.ndarray, sr: int) -> np.ndarray:
        """Compute mel-spectrogram features."""
        if sr != 16000:
            audio = resample(audio, orig_sr=sr, target_sr=16000)

        audio = torch.from_numpy(audio).float()
        mel_transform = T.MelSpectrogram(
            sample_rate=16000,
            n_fft=512,
            win_length=400,
            hop_length=160,
            f_min=0.0,
            f_max=8000.0,
            n_mels=80,
            window_fn=torch.hann_window,
            power=2.0,
            normalized=False
        )
        audio = audio.unsqueeze(0)
        features = mel_transform(audio)
        features = torch.log(features + 1e-9)

        if self.normalize_type == "per_feature":
            mean = features.mean(dim=2, keepdims=True)
            stddev = features.std(dim=2, keepdims=True) + 1e-5
            features = (features - mean) / stddev

        features = features.squeeze(0).permute(1, 0).numpy().astype(np.float32)  # [T, n_mels]
        return features

    def preprocess_audio(self, wav_path: str) -> np.ndarray:
        """Preprocess WAV file to mel-spectrogram features."""
        if not os.path.exists(wav_path):
            raise FileNotFoundError(f"WAV file {wav_path} not found")

        audio, sr = sf.read(wav_path, dtype="float32")
        if audio.ndim > 1:
            audio = audio.mean(axis=1)
        audio = np.concatenate([audio, np.zeros(sr * 2)])  # Add 2s padding
        return self.calculate_audio_features(audio, sr)

    def run_encoder(self, features: np.ndarray) -> np.ndarray:
        """Run encoder on features."""
        features = torch.from_numpy(features).unsqueeze(0).permute(0, 2, 1)  # [1, n_mels, T]
        x_lens = np.array([features.shape[2]], dtype=np.int64)

        encoder_out, _ = self.encoder.run(
            [self.encoder.get_outputs()[0].name, self.encoder.get_outputs()[1].name],
            {
                self.encoder.get_inputs()[0].name: features.numpy(),
                self.encoder.get_inputs()[1].name: x_lens,
            },
        )
        return encoder_out

    def run_decoder(self, token: int, state0: np.ndarray, state1: np.ndarray):
        """Run decoder for a single token."""
        target = np.array([[token]], dtype=np.int32)
        target_len = np.array([1], dtype=np.int32)

        decoder_out, _, state0_next, state1_next = self.decoder.run(
            [
                self.decoder.get_outputs()[0].name,
                self.decoder.get_outputs()[1].name,
                self.decoder.get_outputs()[2].name,
                self.decoder.get_outputs()[3].name,
            ],
            {
                self.decoder.get_inputs()[0].name: target,
                self.decoder.get_inputs()[1].name: target_len,
                self.decoder.get_inputs()[2].name: state0,
                self.decoder.get_inputs()[3].name: state1,
            },
        )
        return decoder_out, state0_next, state1_next

    def run_joiner(self, encoder_out: np.ndarray, decoder_out: np.ndarray) -> np.ndarray:
        """Run joiner to combine encoder and decoder outputs."""
        logit = self.joiner.run(
            [self.joiner.get_outputs()[0].name],
            {
                self.joiner.get_inputs()[0].name: encoder_out,
                self.joiner.get_inputs()[1].name: decoder_out,
            },
        )[0]
        return logit

    def transcribe(self, wav_path: str) -> str:
        """Full transcription pipeline for transducer model."""
        try:
            # Preprocess audio
            features = self.preprocess_audio(wav_path)
            print(f"Features shape: {features.shape}")

            # Initialize decoding
            ans = [self.blank_id]
            state0, state1 = self.get_decoder_state()
            decoder_out, state0_next, state1_next = self.run_decoder(ans[-1], state0, state1)

            # Run encoder
            encoder_out = self.run_encoder(features)
            # print(f"Decoder dimension {decoder_out.shape}")
            # Decode time steps
            for t in range(encoder_out.shape[2]):
                encoder_out_t = encoder_out[:, :, t : t + 1]
                # print(f"Encoder input to jointnet shape {encoder_out_t.shape}")
                # print("="*20)
                # print(f"Original Encoder input to jointnet shape {encoder_out.shape}")
                logits = self.run_joiner(encoder_out_t, decoder_out)
                logits = torch.from_numpy(logits).squeeze()
                idx = torch.argmax(logits, dim=-1).item()
                if idx != self.blank_id:
                    ans.append(idx)
                    state0 = state0_next
                    state1 = state1_next
                    decoder_out, state0_next, state1_next = self.run_decoder(ans[-1], state0, state1)

            # Convert to text
            ans = ans[1:]  # Remove initial blank
            # print(ans[1:])
            tokens = [self.id2token[i+1536] for i in ans]
            text = "".join(tokens).replace("<unk>", "").replace("▁", " ").strip()
            return text
        except Exception as e:
            raise RuntimeError(f"Transcription failed: {str(e)}")

In [21]:
asr = StandaloneASR(os.getcwd())


In [22]:
asr.transcribe("file.wav")

Features shape: (1613, 80)
[170, 179, 177, 52, 22, 146, 105, 170, 177, 168, 177, 205, 167, 76, 67, 151, 35, 175, 64, 10, 112, 121, 175, 209, 183, 32, 4, 67, 83, 181, 119, 83, 12, 44, 170, 179, 177, 52, 46, 155, 72, 59, 29, 118, 43, 178, 173, 5, 6, 132, 167, 10, 118, 197, 64, 76, 170, 164, 32, 9, 7, 76, 67, 172, 52, 2, 177, 162, 150, 39, 115, 21, 197, 67, 79, 161, 14, 180, 179, 59, 5, 170, 30, 32, 5, 6, 132, 167, 22, 1, 173, 26, 161, 207, 167, 100, 61, 167]


'शिवपाल की यह टिप्पणी फिल्म कालिया के डायलॉग से मिलतीजुलती है शिवपाल चाहते हैं कि मुलायम पार्टी के मुखिया फिर से बने फिलहाल सपा अध्यक्ष अखिलेश यादव हैं पिता से पार्टी की कमान छीनी थी'

In [23]:
print(model.transcribe(["file.wav"], batch_size=1, logprobs=False, language_id='hi')[0])

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s][NeMo W 2025-06-18 18:54:02 nemo_logging:393] /content/NeMo/nemo/collections/asr/parts/preprocessing/features.py:417: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
      with torch.cuda.amp.autocast(enabled=False):
    
Transcribing: 100%|██████████| 1/1 [00:05<00:00,  5.38s/it]

[' शिवपाल की यह टिप्पणी फ़िल्म कालिया के डायलॉग से मिलती जुलती है शिवपाल चाहते हैं कि मुलायम पार्टी के मुखिया फिर से बने फिलहाल सपा अध्यक्ष अखिलेश यादव हैं पिता से पार्टी की कमान छीनी थी']


## The output of the transcriptions match. That's good.

## Let's convert them to float 16

In [24]:
from onnxconverter_common import float16
def convert_to_fp16(input_onnx_path, output_onnx_path):
    """Convert ONNX model to FP16."""
    print("Starting FP16 conversion...")
    model = onnx.load(input_onnx_path)
    model_fp16 = float16.convert_float_to_float16(model)
    onnx.save(model_fp16, output_onnx_path)
    print(f"FP16 model saved to {output_onnx_path}")

In [25]:
convert_to_fp16("encoder.onnx", "encoder_fp16.onnx")
convert_to_fp16("decoder.onnx", "decoder_fp16.onnx")
convert_to_fp16("joiner.onnx", "joiner_fp16.onnx")

Starting FP16 conversion...


[NeMo W 2025-06-18 19:11:11 nemo_logging:393] /usr/local/lib/python3.11/dist-packages/onnxconverter_common/float16.py:53: UserWarning: the float32 number -8.581150900965895e-09 will be truncated to -1e-07
      warnings.warn("the float32 number {} will be truncated to {}".format(neg_max, -min_positive_val))
    
[NeMo W 2025-06-18 19:11:11 nemo_logging:393] /usr/local/lib/python3.11/dist-packages/onnxconverter_common/float16.py:53: UserWarning: the float32 number -1.5832484212552345e-09 will be truncated to -1e-07
      warnings.warn("the float32 number {} will be truncated to {}".format(neg_max, -min_positive_val))
    
[NeMo W 2025-06-18 19:11:11 nemo_logging:393] /usr/local/lib/python3.11/dist-packages/onnxconverter_common/float16.py:43: UserWarning: the float32 number 3.1460729132959386e-08 will be truncated to 1e-07
      warnings.warn("the float32 number {} will be truncated to {}".format(pos_min, min_positive_val))
    
[NeMo W 2025-06-18 19:11:11 nemo_logging:393] /usr/local/li

FP16 model saved to encoder_fp16.onnx
Starting FP16 conversion...


[NeMo W 2025-06-18 19:11:32 nemo_logging:393] /usr/local/lib/python3.11/dist-packages/onnxconverter_common/float16.py:53: UserWarning: the float32 number -8.146412966425487e-08 will be truncated to -1e-07
      warnings.warn("the float32 number {} will be truncated to {}".format(neg_max, -min_positive_val))
    
[NeMo W 2025-06-18 19:11:32 nemo_logging:393] /usr/local/lib/python3.11/dist-packages/onnxconverter_common/float16.py:43: UserWarning: the float32 number 5.597248531330479e-08 will be truncated to 1e-07
      warnings.warn("the float32 number {} will be truncated to {}".format(pos_min, min_positive_val))
    


FP16 model saved to decoder_fp16.onnx
Starting FP16 conversion...
FP16 model saved to joiner_fp16.onnx


In [32]:
import os
import json
import numpy as np
import onnxruntime as ort
import soundfile as sf
import torch
import torchaudio
import torchaudio.transforms as T
from librosa import resample
from typing import Tuple

class StandaloneASR:
    def __init__(self, model_dir: str = "model_components"):
        """Initialize the transducer-based ASR system."""
        self.model_dir = model_dir
        self.device = "cuda" if ort.get_device() == "GPU" else "cpu"

        # Ensure model_dir exists
        if not os.path.exists(model_dir):
            raise FileNotFoundError(f"Model directory {model_dir} does not exist")

        # Load ONNX models
        encoder_path = os.path.join(model_dir, "encoder_fp16.onnx")
        decoder_path = os.path.join(model_dir, "decoder_fp16.onnx")
        joiner_path = os.path.join(model_dir, "joiner_fp16.onnx")
        for path in [encoder_path, decoder_path, joiner_path]:
            if not os.path.exists(path):
                raise FileNotFoundError(f"Model file {path} not found")

        session_opts = ort.SessionOptions()
        session_opts.inter_op_num_threads = 1
        session_opts.intra_op_num_threads = 1
        providers = ["CUDAExecutionProvider" if self.device == "cuda" else "CPUExecutionProvider"]

        self.encoder = ort.InferenceSession(encoder_path, sess_options=session_opts, providers=providers)
        self.decoder = ort.InferenceSession(decoder_path, sess_options=session_opts, providers=providers)
        self.joiner = ort.InferenceSession(joiner_path, sess_options=session_opts, providers=providers)

        # Load encoder metadata
        meta = self.encoder.get_modelmeta().custom_metadata_map
        self.normalize_type = meta.get("normalize_type", "")
        self.pred_rnn_layers = int(meta.get("pred_rnn_layers", 2))
        self.pred_hidden = int(meta.get("pred_hidden", 512))

        # Load vocabulary
        vocab_path = os.path.join(model_dir, "tokens.txt")
        if not os.path.exists(vocab_path):
            raise FileNotFoundError(f"Vocab file {vocab_path} not found")
        self.id2token = {}
        with open(vocab_path, encoding="utf-8") as f:
            for line in f:
                t, idx = line.strip().split()
                self.id2token[int(idx)] = t
        self.blank_id = 256  # No_of_vocab - 1  = 257 - 1

    def get_decoder_state(self):
        """Initialize decoder states."""
        batch_size = 1
        state0 = np.zeros((self.pred_rnn_layers, batch_size, self.pred_hidden), dtype=np.float16)
        state1 = np.zeros((self.pred_rnn_layers, batch_size, self.pred_hidden), dtype=np.float16)
        return state0, state1

    def calculate_audio_features(self, audio: np.ndarray, sr: int) -> np.ndarray:
        """Compute mel-spectrogram features."""
        if sr != 16000:
            audio = resample(audio, orig_sr=sr, target_sr=16000)

        audio = torch.from_numpy(audio).float()
        mel_transform = T.MelSpectrogram(
            sample_rate=16000,
            n_fft=512,
            win_length=400,
            hop_length=160,
            f_min=0.0,
            f_max=8000.0,
            n_mels=80,
            window_fn=torch.hann_window,
            power=2.0,
            normalized=False
        )
        audio = audio.unsqueeze(0)
        features = mel_transform(audio)
        features = torch.log(features + 1e-9)

        if self.normalize_type == "per_feature":
            mean = features.mean(dim=2, keepdims=True)
            stddev = features.std(dim=2, keepdims=True) + 1e-5
            features = (features - mean) / stddev

        features = features.squeeze(0).permute(1, 0).numpy().astype(np.float16)  # [T, n_mels]
        return features

    def preprocess_audio_file(self, wav_path: str) -> np.ndarray:
        """Preprocess WAV file to mel-spectrogram features."""
        if not os.path.exists(wav_path):
            raise FileNotFoundError(f"WAV file {wav_path} not found")

        audio, sr = sf.read(wav_path, dtype="float32")
        if audio.ndim > 1:
            audio = audio.mean(axis=1)
        audio = np.concatenate([audio, np.zeros(sr * 2)])  # Add 2s padding
        return audio

    def run_encoder(self, features: np.ndarray) -> np.ndarray:
        """Run encoder on features."""
        features = torch.from_numpy(features).unsqueeze(0).permute(0, 2, 1)  # [1, n_mels, T]
        x_lens = np.array([features.shape[2]], dtype=np.int64)

        encoder_out, _ = self.encoder.run(
            [self.encoder.get_outputs()[0].name, self.encoder.get_outputs()[1].name],
            {
                self.encoder.get_inputs()[0].name: features.numpy(),
                self.encoder.get_inputs()[1].name: x_lens,
            },
        )
        return encoder_out

    def run_decoder(self, token: int, state0: np.ndarray, state1: np.ndarray):
        """Run decoder for a single token."""
        target = np.array([[token]], dtype=np.int32)
        target_len = np.array([1], dtype=np.int32)

        decoder_out, _, state0_next, state1_next = self.decoder.run(
            [
                self.decoder.get_outputs()[0].name,
                self.decoder.get_outputs()[1].name,
                self.decoder.get_outputs()[2].name,
                self.decoder.get_outputs()[3].name,
            ],
            {
                self.decoder.get_inputs()[0].name: target,
                self.decoder.get_inputs()[1].name: target_len,
                self.decoder.get_inputs()[2].name: state0,
                self.decoder.get_inputs()[3].name: state1,
            },
        )
        return decoder_out, state0_next, state1_next

    def run_joiner(self, encoder_out: np.ndarray, decoder_out: np.ndarray) -> np.ndarray:
        """Run joiner to combine encoder and decoder outputs."""
        logit = self.joiner.run(
            [self.joiner.get_outputs()[0].name],
            {
                self.joiner.get_inputs()[0].name: encoder_out,
                self.joiner.get_inputs()[1].name: decoder_out,
            },
        )[0]
        return logit

    def transcribe_file(self, wav_path: str) -> str:
        """Full transcription pipeline for transducer model."""
        try:
            # Preprocess audio
            audio = self.preprocess_audio_file(wav_path)
            return self.transcribe(audio)
        except Exception as e:
            raise RuntimeError(f"Transcription failed: {str(e)}")

    def transcribe(self, audio: np.ndarray) -> str:
        """Full transcription pipeline for transducer model."""
        try:
            # Preprocess audio
            features = self.calculate_audio_features(audio, 16000)
            print(f"Features shape: {features.shape}")

            # Initialize decoding
            ans = [self.blank_id]
            state0, state1 = self.get_decoder_state()
            decoder_out, state0_next, state1_next = self.run_decoder(ans[-1], state0, state1)

            # Run encoder
            encoder_out = self.run_encoder(features)
            # print(f"Decoder dimension {decoder_out.shape}")
            # Decode time steps
            for t in range(encoder_out.shape[2]):
                encoder_out_t = encoder_out[:, :, t : t + 1]
                # print(f"Encoder input to jointnet shape {encoder_out_t.shape}")
                # print("="*20)
                # print(f"Original Encoder input to jointnet shape {encoder_out.shape}")
                logits = self.run_joiner(encoder_out_t, decoder_out)
                logits = torch.from_numpy(logits).squeeze()
                idx = torch.argmax(logits, dim=-1).item()
                if idx != self.blank_id:
                    ans.append(idx)
                    state0 = state0_next
                    state1 = state1_next
                    decoder_out, state0_next, state1_next = self.run_decoder(ans[-1], state0, state1)

            # Convert to text
            ans = ans[1:]  # Remove initial blank
            # print(ans[1:])
            tokens = [self.id2token[i+1536] for i in ans]
            text = "".join(tokens).replace("<unk>", "").replace("▁", " ").strip()
            return text
        except Exception as e:
            raise RuntimeError(f"Transcription failed: {str(e)}")

In [33]:
asr = StandaloneASR(os.getcwd())

In [34]:
print(asr.transcribe_file("file.wav"))

Features shape: (2419, 80)
शिवपाल की यह टिप्पणी फिल्म कालिया के डायलॉग से मिलतीजुलती है शिवपाल चाहते हैं कि मुलायम पार्टी के मुखिया फिर से बने फिलहाल सपा अध्यक्ष अखिलेश यादव हैं पिता से पार्टी की कमान छीनी थी
